In [52]:
import multiprocessing
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, f1_score, roc_auc_score

In [53]:
X_train= pd.read_csv("data/X_train.csv", encoding='cp949')
y_train= pd.read_csv("data/y_train.csv", encoding='cp949')
X_test= pd.read_csv("data/X_test.csv", encoding='cp949')

In [54]:
X_train.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


In [55]:
y_train.gender = y_train.gender.apply(lambda x : '여자' if x ==0 else '남자')
y_train['gender'] = y_train.loc[:,'gender'].apply(lambda x: 1 if x=='남자' else 0)
y_train.head(50)

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,1


In [56]:
y_train.iloc[:,1] = y_train.gender.map({1:'male', 0:'female'})

In [57]:
y_train

,cust_id,gender
0,0,female
1,1,female
2,2,male
3,3,male
4,4,female
...,...,...
3495,3495,male
3496,3496,male
3497,3497,female
3498,3498,female


In [35]:
y_train = y_train.iloc[:,1]
y_train

0       0
1       0
2       1
3       1
4       0
       ..
3495    1
3496    1
3497    0
3498    0
3499    0
Name: gender, Length: 3500, dtype: int64

In [58]:
X_train.loc[:,'환불금액'].fillna(0, inplace=True)
X_train.환불금액

0       6860000.0
1        300000.0
2             0.0
3             0.0
4             0.0
          ...    
3495          0.0
3496    6049600.0
3497          0.0
3498          0.0
3499    5973000.0
Name: 환불금액, Length: 3500, dtype: float64

In [29]:
pd.get_dummies(X_train['주구매상품'])

,가공식품,가구,건강식품,골프,구두,기타,남성 캐주얼,남성 트랜디,남성정장,농산물,...,주방용품,차/커피,축산가공,침구/수예,캐주얼,커리어,통신/컴퓨터,트래디셔널,피혁잡화,화장품
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3497,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [28]:
pd.get_dummies(X_train['주구매지점'])

,강남점,관악점,광주점,노원점,대구점,대전점,동래점,미아점,본 점,부산본점,...,안양점,영등포점,울산점,인천점,일산점,잠실점,전주점,창원점,청량리점,포항점
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3496,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3498,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
